In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
# /content/drive/MyDrive/machine_learning/data/breast-cancer-wisconsin.data
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


missing_values = ["?"]

df = pd.read_csv("/content/drive/MyDrive/machine_learning/data/breast-cancer-wisconsin.data", na_values=missing_values)
# df = pd.read_csv("/content/drive/MyDrive/machine_learning/data/breast-cancer-wisconsin.data")
columns = [
'Simple code number',
'Clump Thickness',
'Uniformity of Cell Size',
'Uniformity of Cell Shape',
'Marginal Adhesion',
'Single Epithelial Cell Size',
'Bare Nuclei',
'Bland Chromatin',
'Normal Nucleoli',
'Mitoses',
'Class'
]
df.columns = columns
print(df)


     Simple code number  Clump Thickness  Uniformity of Cell Size  \
0               1002945                5                        4   
1               1015425                3                        1   
2               1016277                6                        8   
3               1017023                4                        1   
4               1017122                8                       10   
..                  ...              ...                      ...   
693              776715                3                        1   
694              841769                2                        1   
695              888820                5                       10   
696              897471                4                        8   
697              897471                4                        8   

     Uniformity of Cell Shape  Marginal Adhesion  Single Epithelial Cell Size  \
0                           4                  5                            7   
1        

In [120]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Simple code number           698 non-null    int64  
 1   Clump Thickness              698 non-null    int64  
 2   Uniformity of Cell Size      698 non-null    int64  
 3   Uniformity of Cell Shape     698 non-null    int64  
 4   Marginal Adhesion            698 non-null    int64  
 5   Single Epithelial Cell Size  698 non-null    int64  
 6   Bare Nuclei                  682 non-null    float64
 7   Bland Chromatin              698 non-null    int64  
 8   Normal Nucleoli              698 non-null    int64  
 9   Mitoses                      698 non-null    int64  
 10  Class                        698 non-null    int64  
dtypes: float64(1), int64(10)
memory usage: 60.1 KB
None


In [121]:
df.isna().sum()

Simple code number              0
Clump Thickness                 0
Uniformity of Cell Size         0
Uniformity of Cell Shape        0
Marginal Adhesion               0
Single Epithelial Cell Size     0
Bare Nuclei                    16
Bland Chromatin                 0
Normal Nucleoli                 0
Mitoses                         0
Class                           0
dtype: int64

In [122]:
df.dropna(axis=0, inplace = True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 682 entries, 0 to 697
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Simple code number           682 non-null    int64  
 1   Clump Thickness              682 non-null    int64  
 2   Uniformity of Cell Size      682 non-null    int64  
 3   Uniformity of Cell Shape     682 non-null    int64  
 4   Marginal Adhesion            682 non-null    int64  
 5   Single Epithelial Cell Size  682 non-null    int64  
 6   Bare Nuclei                  682 non-null    float64
 7   Bland Chromatin              682 non-null    int64  
 8   Normal Nucleoli              682 non-null    int64  
 9   Mitoses                      682 non-null    int64  
 10  Class                        682 non-null    int64  
dtypes: float64(1), int64(10)
memory usage: 63.9 KB
None


In [123]:
df.drop(["Simple code number"], axis=1, inplace=True)
# Change target(Class) value (2 -> 0 / 4 -> 1)
df.at[df[df['Class'] == 2].index, 'Class'] = 0
df.at[df[df['Class'] == 4].index, 'Class'] = 1
# Split feature and target data
X = pd.DataFrame(df.iloc[:,0:9], dtype=np.dtype("int64"))
y = df.iloc[:,9]
## Find Best model and options
# Run findBestOptions()
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.7, shuffle=True)



In [124]:
def decision_tree_gini(scalers, X, y):
  params = {'criterion':['gini'],'splitter':['best','random'], 'max_depth':[2,3,4,5,6,7,8],'min_samples_split':[2,3,4,5]}
  dtc = DecisionTreeClassifier()
  return_list = []
  for scaler in scalers:
    X = scaler.fit_transform(X)
    for i in range(2,11):
      gd_sr = GridSearchCV(estimator = dtc, param_grid = params, cv=i, n_jobs=-1)
      gd_sr.fit(X,y)
      temp_list = []
      temp_list.append(gd_sr.best_score_)
      temp_list.append("decision tree with gini")
      temp_list.append(gd_sr.best_params_)
      temp_model = DecisionTreeClassifier(criterion='gini', splitter=gd_sr.best_params_['splitter'], max_depth=gd_sr.best_params_['max_depth'], min_samples_split=gd_sr.best_params_['min_samples_split'])
      temp_model.fit(X,y)
      temp_list.append(temp_model)
      temp_list.append(scaler)
      return_list.append(temp_list)

  return return_list


In [125]:
def find_Scale_Model(X,y):
  global best_score,best_cv,best_scaler,best_model
  best_score=-1.0
  DecisionTreeClassifier_criterion=["gini","entropy"]

  LogisticRegression_solver=["lbfgs","newton-cg","liblinear","sag","saga"]

  SVC_kernel=["rbf","poly","sigmoid","linear"]

  gamma_list=[0.001,0.01,0.1,1,10]
  # result list foramt = [점수, ]
  result_list = []
  cv_k=[2,3,4,5,6,7,8,9,10]
  scalers=[StandardScaler(), MinMaxScaler()]
  gini_result = decision_tree_gini(scalers, X, y)
  print(gini_result)
  result_list += gini_result
  result_list.sort(key = lambda i : i[0], reverse = True)


  return result_list

In [126]:
result = find_Scale_Model(train_X, train_y)

[[0.9656862745098038, 'decision tree with gini', {'criterion': 'gini', 'max_depth': 4, 'min_samples_split': 5, 'splitter': 'random'}, DecisionTreeClassifier(max_depth=4, min_samples_split=5, splitter='random'), StandardScaler()], [0.9607843137254902, 'decision tree with gini', {'criterion': 'gini', 'max_depth': 4, 'min_samples_split': 3, 'splitter': 'random'}, DecisionTreeClassifier(max_depth=4, min_samples_split=3, splitter='random'), StandardScaler()], [0.9656862745098039, 'decision tree with gini', {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2, 'splitter': 'random'}, DecisionTreeClassifier(max_depth=3, splitter='random'), StandardScaler()], [0.9609756097560975, 'decision tree with gini', {'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 2, 'splitter': 'best'}, DecisionTreeClassifier(max_depth=2), StandardScaler()], [0.9705882352941176, 'decision tree with gini', {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2, 'splitter': 'random'}, DecisionTreeC

In [127]:
for i in range(5):
  print(result[i])

[0.9710144927536233, 'decision tree with gini', {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 3, 'splitter': 'random'}, DecisionTreeClassifier(max_depth=3, min_samples_split=3, splitter='random'), MinMaxScaler()]
[0.9705882352941176, 'decision tree with gini', {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2, 'splitter': 'random'}, DecisionTreeClassifier(max_depth=3, splitter='random'), StandardScaler()]
[0.9705882352941176, 'decision tree with gini', {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2, 'splitter': 'random'}, DecisionTreeClassifier(max_depth=3, splitter='random'), MinMaxScaler()]
[0.9705769230769231, 'decision tree with gini', {'criterion': 'gini', 'max_depth': 4, 'min_samples_split': 2, 'splitter': 'random'}, DecisionTreeClassifier(max_depth=4, splitter='random'), StandardScaler()]
[0.9705753184014054, 'decision tree with gini', {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 5, 'splitter': 'random'}, DecisionTreeClassifier

In [128]:
X = result[0][4].fit_transform(X)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.7, shuffle=True)
model = result[0][3].fit(train_X, train_y)
print("Model score: ", end="")
print(model.score(test_X, test_y))

Model score: 0.9225941422594143
